In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pydot
import cv2
import time
from PIL import Image, ImageDraw

In [2]:
def log_mse(y_true, y_pred):
    return tf.reduce_mean(tf.math.log1p(tf.math.squared_difference(y_pred, y_true)), axis=-1)

def focal_loss(alpha=0.9, gamma=2):
  def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
    weight_a = alpha * (1 - y_pred) ** gamma * targets
    weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
    
    return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b

  def loss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))

    loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)

    return tf.reduce_mean(loss)

  return loss

In [3]:
model = tf.keras.models.load_model(r'C:\Users\densh\Desktop\catdogs\TestTask\Custom_test\CustomModel')

In [4]:
def IOUab(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [5]:
def IOUhw(bbox1, bbox2):
    '''Calculate overlap between two bounding boxes [x, y, w, h] as the area of intersection over the area of unity'''
    x1, y1, w1, h1 = bbox1[0], bbox1[1], bbox1[2], bbox1[3]
    x2, y2, w2, h2 = bbox2[0], bbox2[1], bbox2[2], bbox2[3]

    w_I = max(x1-w1/2,x2-w2/2) - min(x1+w1/2,x2+w2/2)
    h_I = max(y1-h1/2,y2-h2/2) - min(y1+h1/2,y2+h2/2)
    if w_I <= 0 or h_I <= 0:  # no overlap
        return 0.
    I = w_I * h_I

    U = w1 * h1 + w2 * h2 - I

    return I / U

In [6]:
validation_data_dir = r'C:\Users\densh\Desktop\catdogs\keras_try\model_testing\valid_set'
Transfer_test = r'C:\Users\densh\Desktop\catdogs\TestTask\Custom_test'
counter = 0
overall_time = 0
mIoU = 0
prediction_count = 0
for image in glob.glob(os.path.join(validation_data_dir,'*.jpg')):
    title, ext = os.path.splitext(os.path.basename(image))
    txt_file_name = title + '.txt'
    imag = cv2.imread(image)
    H, W, _  = imag.shape
    img = cv2.resize(imag,(440,440))    
    img = img.reshape(1,440,440,3)
    img = np.array(img)/255.0
    tic = time.perf_counter()
    results = model.predict(img)
    toc = time.perf_counter()
    #print(f"1 inference file in {toc - tic:0.4f} seconds")
    timer = toc - tic
    overall_time = overall_time + timer

    bbox1 = []
    bbox2 = []

    label_names = ['cat','dog']
    class_label = label_names[np.argmax(results[0])]

    x1_, y1_, x2_, y2_ = results[1][:][0]

    x1 = int(W * (x1_ - x2_/2))
    x2 = int(W * (x1_ + x2_/2))
    y1 = int(H * (y1_ - y2_/2))
    y2 = int(H * (y1_ + y2_/2))
    bbox1 = [x1,y1,x2,y2]
    top_left_pred = (x1,y1)
    right_bottom_pred = (x2, y2)

    with open(os.path.join(Transfer_test+'/Labels'+'/',title + '_pred' + '.txt'),'w+') as new_txt:
        new_txt.write(str(class_label) + '\t' + str(x1) 
        + '\t' + str(y1) + '\t' + str(x2) + '\t' + str(y2) + '\t' + str(timer))
    
    truth_txt = open(os.path.join(validation_data_dir,txt_file_name),'r')
    labelList = truth_txt.readlines()
    for label in labelList:
        label = label.strip().split()
        class_=int(label[0])
        x1 = int(label[1])
        x2 = int(label[3])
        y1 = int(label[2])
        y2 = int(label[4])
    
    top_left_true = (x1, y1)
    right_bottom_true = (x2,y2)
    bbox2 = [x1,y1,(x2),y2]

    IoU = float(IOUab(bbox1, bbox2)*100)

    if class_label == label_names[class_-1]:
        prediction_count =  prediction_count + 1
    counter = counter + 1

    cv2.rectangle(imag, top_left_true, right_bottom_true,[0,0,0],2)
    cv2.rectangle(imag, top_left_pred, right_bottom_pred,[255,0,0],2)
    cv2.putText( imag,'IoU = '+f"{IoU:.{2}f}" +' '+  label_names[class_-1], (int(x1), int(y1) + int(28)),  cv2.FONT_HERSHEY_TRIPLEX, 0.8, [0, 0, 0], 2)
    cv2.imwrite(r'C:\Users\densh\Desktop\catdogs\TestTask\Custom_test\Images/' + title+'.jpg',imag)
    mIoU = mIoU + IoU
    #break
mIoU = mIoU/counter
mean_time = overall_time/counter
Acc = prediction_count/counter
print(mIoU, mean_time, Acc)






32.500543604354526 0.058938416539409684 0.8804071246819338
